In [1]:
import pandas as pd
import glob
import soundfile
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
data_path='drive/MyDrive/255 Project/Project'

In [13]:
# os.listdir('drive/MyDrive/255 Project/Project/Data/AudioWAV/1091_DFA_ANG_XX.wav')

In [2]:
import os

In [20]:
import librosa
path='drive/MyDrive/255 Project/Project/Data/AudioWAV/1091_DFA_ANG_XX.wav'
audio_data, sr = librosa.load(path, sr=None)

In [21]:
librosa.get_samplerate(path)

16000

In [16]:
audio_data.shape

# Audio file is mono
# native sampling rate
#  if the audio file is a mono WAV file with a length of 10 seconds 
# and a sampling rate of 44100 Hz, the audio_data array might have a shape of (441000,)
# Shape represents # of audio samples at a particular point in time
# each sample represents the amplitude

(32032,)

In [22]:
import librosa
path='drive/MyDrive/255 Project/Project/Data/AudioWAV/1091_DFA_ANG_XX.wav'
audio_data, sr = librosa.load(path, sr=10000)

In [ ]:
librosa.get_samplerate(audio_file)

In [23]:
audio_data.shape

(20020,)

In [ ]:
audio_data.min(),audio_data.max()

(-0.6310425, 0.62527466)

In [ ]:
path=r"drive/MyDrive/255 Project/Project/Data/AudioWAV/1001_DFA_ANG_XX.wav"
X = librosa.load(path, sr=None)
audio_data,sample_rate=librosa.load(path, sr=None)

len(librosa.stft(X[0]))
# len(X[0])

1025

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    Y = librosa.load(file_name, sr=None,mono=True)
    audio_data,sample_rate=librosa.load(file_name, sr=None)
    X=Y[0]
    if chroma:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel)) 
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '1':'neutral',
  '2':'calm',
  '3':'happy',
  '4':'sad',
  '5':'angry',
  '6':'fearful',
  '7':'disgust',
  '8':'surprised'
}

#Emotions to observe
observed_emotions=['neutral' , 'calm','happy', 'sad','angry','fearful','disgust','surprised']


In [ ]:
import os
# os.listdir('drive/MyDrive/255 Project/Project/Data/AudioWAV')

In [ ]:
# glob.glob("drive/MyDrive/255 Project/Project/Data/AudioWAV/*.wav")

In [ ]:
import pandas as pd
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(r"drive/MyDrive/255 Project/Project/Data/AudioWAV/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("_")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    y=np.array(pd.get_dummies(y))
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)
# mfcc-> 40
# chroma->12
# Mel->128

In [ ]:
import numpy as np
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

/usr/local/lib/python3.9/dist-packages/librosa/core/pitch.py:102: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [ ]:
x_train.shape

(5581, 180)

In [ ]:
y_train

array([[0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]], dtype=uint8)

In [ ]:
x_test.shape

(1861, 180)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Define the model architecture
# input_shape = (None,x_train.shape[1])  # 40 MFCCs
input_shape = (3, 60)
num_classes = 6  # Number of emotions
# Define the LSTM model
model = keras.Sequential()
model.add(layers.LSTM(units=128, return_sequences=True, input_shape=input_shape))
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(units=128, return_sequences=True))
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(units=64, return_sequences=True))
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(units=64))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(units=64, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(units=num_classes, activation="softmax"))


In [ ]:
# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

# Print the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 3, 128)            96768     
                                                                 
 dropout_5 (Dropout)         (None, 3, 128)            0         
                                                                 
 lstm_5 (LSTM)               (None, 3, 128)            131584    
                                                                 
 dropout_6 (Dropout)         (None, 3, 128)            0         
                                                                 
 lstm_6 (LSTM)               (None, 3, 64)             49408     
                                                                 
 dropout_7 (Dropout)         (None, 3, 64)             0         
                                                                 
 lstm_7 (LSTM)               (None, 64)               

In [ ]:
# Reshape the input data to have a time step dimension of 1
x_train = x_train.reshape((-1, 3, 60))
x_test = x_test.reshape((-1, 3, 60))
# x_test=x_test.reshape((-1, 1, 180))
x_train.shape

(5581, 3, 60)

In [ ]:
x_test.shape

(1861, 3, 60)

In [ ]:
y_train.shape

(5581, 6)

In [ ]:
model.fit(x_train,y_train,batch_size=20,epochs=200,validation_data=(x_test,y_test))

Epoch 1/200
280/280 [==============================] - 13s 15ms/step - loss: 1.6725 - accuracy: 0.2569 - val_loss: 1.5309 - val_accuracy: 0.3353
Epoch 2/200
280/280 [==============================] - 3s 10ms/step - loss: 1.5444 - accuracy: 0.3464 - val_loss: 1.4757 - val_accuracy: 0.3933
Epoch 3/200
280/280 [==============================] - 3s 12ms/step - loss: 1.5077 - accuracy: 0.3688 - val_loss: 1.4711 - val_accuracy: 0.3810
Epoch 4/200
280/280 [==============================] - 4s 13ms/step - loss: 1.4938 - accuracy: 0.3919 - val_loss: 1.4428 - val_accuracy: 0.4035
Epoch 5/200
280/280 [==============================] - 3s 11ms/step - loss: 1.4760 - accuracy: 0.3965 - val_loss: 1.4541 - val_accuracy: 0.3923
Epoch 6/200
280/280 [==============================] - 3s 10ms/step - loss: 1.4673 - accuracy: 0.4003 - val_loss: 1.4413 - val_accuracy: 0.3998
Epoch 7/200
280/280 [==============================] - 3s 12ms/step - loss: 1.4591 - accuracy: 0.3985 - val_loss: 1.4486 - val_accuracy

In [ ]:
model.predict(x_test)

59/59 [==============================] - 1s 4ms/step


array([[4.76798229e-02, 2.90296674e-01, 1.51015725e-02, 8.20355117e-02,
        5.29506862e-01, 3.53795700e-02],
       [2.09976966e-03, 8.02423747e-04, 9.90351498e-01, 6.19304739e-03,
        1.73232311e-05, 5.35915256e-04],
       [4.11907036e-04, 4.69607796e-04, 3.10941832e-05, 9.99087453e-01,
        7.59178331e-10, 9.90534305e-15],
       ...,
       [2.11965777e-02, 6.20172501e-01, 1.02835502e-02, 2.83842683e-01,
        6.04856536e-02, 4.01910068e-03],
       [3.90839158e-03, 6.67309165e-02, 2.56518096e-01, 1.52916564e-02,
        7.92658627e-02, 5.78285098e-01],
       [3.89772654e-03, 1.28053771e-02, 2.11407896e-02, 9.62151110e-01,
        4.91319543e-06, 5.25340376e-08]], dtype=float32)

In [ ]:
# feature importance using rf
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Load the extracted features from the .npy file
features = np.load('features.npy')

# Load the corresponding labels for the features
labels = np.load('labels.npy')

# Split the data into training and testing sets
split_idx = int(len(features) * 0.8)
train_features, train_labels = features[:split_idx], labels[:split_idx]
test_features, test_labels = features[split_idx:], labels[split_idx:]

# Train a Random Forest model on the training data
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_features, train_labels)

# Extract feature importance from the trained model
feature_importance = rf_model.feature_importances_


In [ ]:
# Machine learning classifier
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# load the saved model's weights
model_weights = np.load('model_weights.npy')

# load the training data and labels
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')

# define the random forest classifier
rf = RandomForestClassifier()

# fit the random forest on the training data
rf.fit(X_train, y_train)

# set the weights of the random forest to the saved model weights
rf.estimators_ = model_weights

# load the test data
X_test = np.load('X_test.npy')

# make predictions on the test data using the random forest with the saved model weights
y_pred = rf.predict(X_test)
